In [1]:
# All imports and setup
import pandas as pd
from sqlalchemy import create_engine, text
from tqdm.notebook import tqdm
tqdm.pandas()


# Add project root to path 
import sys
from pathlib import Path
sys.path.append(str(Path("../").resolve()))



In [2]:
import src.preprocessing_utils as prep
from src.db_utils import get_db_credentials

In [11]:
# Connect to PostgreSQL via SQLAlchemy
creds = get_db_credentials()
conn_str = f"postgresql+psycopg2://{creds['user']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['db_name']}"
engine = create_engine(conn_str)

# Load tickets (skip demo for pipeline demo purposes)
query = "SELECT ticket_id, subject, body FROM tickets WHERE demo_flag = FALSE;"
tickets_df = pd.read_sql(query, engine)

tickets_df.head()

,ticket_id,subject,body
0,TKT-500000,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge..."
1,TKT-500001,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g..."
2,TKT-500002,Can’t pay online — keeps declining 😕,"Hi team, Hi, I'm Jamie Salonen. Not sure if th..."
3,TKT-500003,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble..."
4,TKT-500004,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on..."


In [ ]:
#  Preprocessing serial : Detect language and clean text
def preprocess_ticket_row(row):
    """
    Apply language detection and cleaning for a single row.
    Translation, PII masking, keywords handled in batch later.
    """
    subject = row.get("subject", "")
    body = row.get("body", "")

    lang = prep.detect_language(body)
    subject_clean = prep.clean_text(subject)
    body_clean = prep.clean_text(body)

    return {
        "ticket_id": row["ticket_id"],
        "lang": lang,
        "subject_clean": subject_clean,
        "body_clean": body_clean
    }




In [ ]:
# Apply row-wise detect language and cleaning
tickets_cleaned = tickets_df.progress_apply(preprocess_ticket_row, axis=1)
tickets_cleaned = pd.DataFrame(list(tickets_cleaned))
tickets_cleaned.head()

In [ ]:
# Initialize columns for translated text
tickets_cleaned["subject_translated"] = ""
tickets_cleaned["body_translated"] = ""

# Batch translate by language 

# Only languages with translation models will be processed here
for lang in prep.TRANSLATION_MODELS.keys():
    # Get indices of rows with this language
    idx = tickets_cleaned[tickets_cleaned["lang"] == lang].index
    if len(idx) == 0:
        continue  # skip if no rows in this language

    # Prepare text batches
    subjects = tickets_cleaned.loc[idx, "subject_clean"].tolist()
    bodies = tickets_cleaned.loc[idx, "body_clean"].tolist()

    # Translate in batch using preprocessing.py functions
    tickets_cleaned.loc[idx, "subject_translated"] = prep.translate_texts_batch(subjects, lang)
    tickets_cleaned.loc[idx, "body_translated"] = prep.translate_texts_batch(bodies, lang)

    # Log batch info
    print(f"[INFO] Translated batch of {len(idx)} tickets for language '{lang}'")

# Fill English/unknown rows with original cleaned content 
mask = tickets_cleaned["subject_translated"] == ""
tickets_cleaned.loc[mask, "subject_translated"] = tickets_cleaned.loc[mask, "subject_clean"]
tickets_cleaned.loc[mask, "body_translated"] = tickets_cleaned.loc[mask, "body_clean"]

# Verify
tickets_cleaned.head()

In [ ]:
# PII masking
tickets_cleaned["subject_masked"] = tickets_cleaned["subject_translated"].progress_apply(prep.mask_pii_en)
tickets_cleaned["body_masked"] = tickets_cleaned["body_translated"].progress_apply(prep.mask_pii_en)

# Verify
tickets_cleaned.head()


In [ ]:
tickets_cleaned.head(20)

In [ ]:
tickets_cleaned.to_csv("tickets_cleaned.csv", index=False)

In [3]:

# Load from CSV into DataFrame
tickets_cleaned = pd.read_csv("tickets_cleaned.csv")

# Preview
tickets_cleaned.head()


,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting..."
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti..."
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ..."


In [4]:
# Combine text columns into one for keyword extraction and later use
tickets_cleaned["combined_text"] = tickets_cleaned["subject_masked"] + "|| " + tickets_cleaned["body_masked"]

# Verify
tickets_cleaned.head()


,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked,combined_text
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting...",URGENT: Hitting rate limits on /transactions (...
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti...",Question about KYC — documents keep getting re...
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is...","Can’t pay online — keeps declining|| Hi team, ..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ...","PRIO: Card missing, block now|| Hello, Hi, I'm..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ...","Tokenization failed during wallet setup|| Hey,..."


In [5]:
# Extract keywords from the combined text
keywords_list = prep.extract_keywords_batch(tickets_cleaned["combined_text"].tolist(), top_n=10)

# Store keywords as a comma-separated string
tickets_cleaned["keywords"] = [", ".join(kws) for kws in keywords_list]

# Verify
tickets_cleaned.head()


Extracting keywords:   2%|▎              | 1003/49800 [00:33<24:26, 33.28text/s]

[INFO] Processed 1000/49800 texts


Extracting keywords:   4%|▌              | 2005/49800 [01:02<23:25, 34.01text/s]

[INFO] Processed 2000/49800 texts


Extracting keywords:   6%|▉              | 3003/49800 [01:32<24:05, 32.37text/s]

...

Extracting keywords:  98%|█████████████▊| 49005/49800 [24:24<00:24, 32.64text/s]

[INFO] Processed 49000/49800 texts


Extracting keywords: 100%|██████████████| 49800/49800 [24:48<00:00, 33.46text/s]


[INFO] Keyword extraction completed for 49800 texts


,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked,combined_text,keywords
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting...",URGENT: Hitting rate limits on /transactions (...,"rate limits transactions error, limits transac..."
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti...",Question about KYC — documents keep getting re...,"kyc documents getting rejected, kyc documents ..."
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is...","Can’t pay online — keeps declining|| Hi team, ...","pay online keeps declining, online keeps decli..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ...","PRIO: Card missing, block now|| Hello, Hi, I'm...","prio card missing block, prio card missing, hi..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ...","Tokenization failed during wallet setup|| Hey,...","tokenization failed wallet, tokenization faile..."


In [ ]:
# # Hello, Hi, I'm Sam Heikkinen. My card keeps getting errors when I try to pay. Balance should be fine, so I'm confused. I tried three times today, same result. Balance shkouldj be fine, so I'm confused.
# # Hallå, Hi, I'm Henrik Svensson. Jag har problem med card lost/stolen. Som jag nämnde tidigare, detta blockerar oss. Phone: +46 101 389 4079 IBAN: FI9106024211679678 Card: 4000 1234 5678 9010 Säg till om ni behöver skärmdumpar. Med vänlig hälsning.
# # Hei, Hi, I'm Sanna Mäkinen. Ongelma liittyy aiheeseen: API Rate Limit/Auth. 💳 IBAN: FI8663695284994003 As mentioned earlier, this is blocking our checkout. Ilmoittakaa jos tarvitsette kuvakaappauksia. Saldon pitäisi riittää, joten olen hämmentynyt. Ilmoittakaa jos tarvitsette kuvakaappauksia. Saldon pitäisi riittää, joten olen hämmentynyt.
# #Hej, Hi, I'm Matilda Virtanen. Jag har problem med dispute/chargeback. Testade också på en annan enhet. 🙏 Phone: +358 167 297 3630 Email: matilda.virtanen@example.com Card: 4970 **** **** 8430 Säg till om ni behöver skärmdumpar. Försökte tre gånger idag, samma resultat. Försökte tre gånger idag, samma resultat. Täckningen borde räcka, så jag är förvirrad. Tack!

# masked = preprocess_ticket("Hej, Hi, I'm Matilda Virtanen. Jag har problem med dispute/chargeback. Testade också på en annan enhet. 🙏 Phone: +358 167 297 3630 Email: matilda.virtanen@example.com Card: 4970 **** **** 8430 Säg till om ni behöver skärmdumpar. Försökte tre gånger idag, samma resultat. Försökte tre gånger idag, samma resultat. Täckningen borde räcka, så jag är förvirrad. Tack!")

In [6]:
tickets_cleaned.head()

,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked,combined_text,keywords
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting...",URGENT: Hitting rate limits on /transactions (...,"rate limits transactions error, limits transac..."
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti...",Question about KYC — documents keep getting re...,"kyc documents getting rejected, kyc documents ..."
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is...","Can’t pay online — keeps declining|| Hi team, ...","pay online keeps declining, online keeps decli..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ...","PRIO: Card missing, block now|| Hello, Hi, I'm...","prio card missing block, prio card missing, hi..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ...","Tokenization failed during wallet setup|| Hey,...","tokenization failed wallet, tokenization faile..."


In [19]:
# Combine text columns into one for keyword extraction and later use
tickets_cleaned["combined_text"] = tickets_cleaned["subject_translated"] + "|| " + tickets_cleaned["body_masked"]

# Verify
tickets_cleaned.head(10)

,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked,combined_text,keywords
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting...",URGENT: Hitting rate limits on /transactions (...,"rate limits transactions error, limits transac..."
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti...",Question about KYC — documents keep getting re...,"kyc documents getting rejected, kyc documents ..."
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is...","Can’t pay online — keeps declining|| Hi team, ...","pay online keeps declining, online keeps decli..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ...","PRIO: Card missing, block now|| Hello, Hi, I'm...","prio card missing block, prio card missing, hi..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ...","Tokenization failed during wallet setup|| Hey,...","tokenization failed wallet, tokenization faile..."
5,TKT-500005,en,"FYI: Sandbox ok, prod failing","Hello, Hi, I'm Jordan Mäkinen. I'm having trou...","FYI: Sandbox ok, prod failing","Hello, Hi, I'm Jordan Mäkinen. I'm having trou...","FYI: [PERSON], prod failing","Hello, Hi, I'm [PERSON]. I'm having trouble wi...","FYI: Sandbox ok, prod failing|| Hello, Hi, I'm...","trouble integration sdk launch, trouble integr..."
6,TKT-500006,fi,PRIO: Miten kasvattaa API-kiintiötä?,"Hei, Hi, I'm Sanna Mäkinen. Ongelma liittyy ai...",PRIO: How to increase the API quota?,"Hi, Hi, I'm Sanna Mäkinen. The problem relates...",PRIO: How to increase the API quota?,"Hi, Hi, I'm [PERSON]. The problem relates to: ...","PRIO: How to increase the API quota?|| Hi, Hi,...","prio increase api quota, increase api quota hi..."
7,TKT-500007,en,Declined transactions without reason?,"Hello, Hi, I'm Drew Laine. My card keeps getti...",Declined transactions without reason?,"Hello, Hi, I'm Drew Laine. My card keeps getti...",Declined transactions without reason?,"Hello, Hi, I'm [PERSON]. My card keeps getting...","Declined transactions without reason?|| Hello,...","card keeps getting declined, getting declined ..."
8,TKT-500008,sv,Tillfällig höjning av e‑handelsgräns,"Hej där, Hi, I'm Sofia Ahonen. Mitt kort blir ...",Temporary increase of e-commerce limit,"Hi there, Hi, I'm Sofia Ahonen. My card gets w...",Temporary increase of e-commerce limit,"Hi there, Hi, I'm [PERSON]. My card gets wrong...",Temporary increase of e-commerce limit|| Hi th...,"temporary increase commerce limit, increase co..."
9,TKT-500009,en,Question about last month’s fees,"Hi, I'm Alex Ahonen. Not sure if this is on my...",Question about last month’s fees,"Hi, I'm Alex Ahonen. Not sure if this is on my...",Question about last month’s fees,"Hi, I'm [PERSON]. Not sure if this is on my si...","Question about last month’s fees|| Hi, I'm [PE...","card 4970 7040 tes

In [21]:

import re

# Define greetings to remove (case-insensitive)
greetings_pattern = r'\b(hi|hello|hey)\b'

# Remove greetings from combined_text
tickets_cleaned["combined_text"] = tickets_cleaned["combined_text"].apply(
    lambda x: re.sub(greetings_pattern, '', x, flags=re.IGNORECASE)
)

# Remove extra whitespace after removal
tickets_cleaned["combined_text"] = tickets_cleaned["combined_text"].str.replace(r'\s+', ' ', regex=True).str.strip()

# Verify
tickets_cleaned.head(10)


,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked,combined_text,keywords
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting...",URGENT: Hitting rate limits on /transactions (...,"rate limits transactions error, limits transac..."
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti...",Question about KYC — documents keep getting re...,"kyc documents getting rejected, kyc documents ..."
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is...","Can’t pay online — keeps declining|| team, , I...","pay online keeps declining, online keeps decli..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ...","PRIO: Card missing, block now|| , , I'm [PERSO...","prio card missing block, prio card missing, hi..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ...","Tokenization failed during wallet setup|| , , ...","tokenization failed wallet, tokenization faile..."
5,TKT-500005,en,"FYI: Sandbox ok, prod failing","Hello, Hi, I'm Jordan Mäkinen. I'm having trou...","FYI: Sandbox ok, prod failing","Hello, Hi, I'm Jordan Mäkinen. I'm having trou...","FYI: [PERSON], prod failing","Hello, Hi, I'm [PERSON]. I'm having trouble wi...","FYI: Sandbox ok, prod failing|| , , I'm [PERSO...","trouble integration sdk launch, trouble integr..."
6,TKT-500006,fi,PRIO: Miten kasvattaa API-kiintiötä?,"Hei, Hi, I'm Sanna Mäkinen. Ongelma liittyy ai...",PRIO: How to increase the API quota?,"Hi, Hi, I'm Sanna Mäkinen. The problem relates...",PRIO: How to increase the API quota?,"Hi, Hi, I'm [PERSON]. The problem relates to: ...","PRIO: How to increase the API quota?|| , , I'm...","prio increase api quota, increase api quota hi..."
7,TKT-500007,en,Declined transactions without reason?,"Hello, Hi, I'm Drew Laine. My card keeps getti...",Declined transactions without reason?,"Hello, Hi, I'm Drew Laine. My card keeps getti...",Declined transactions without reason?,"Hello, Hi, I'm [PERSON]. My card keeps getting...","Declined transactions without reason?|| , , I'...","card keeps getting declined, getting declined ..."
8,TKT-500008,sv,Tillfällig höjning av e‑handelsgräns,"Hej där, Hi, I'm Sofia Ahonen. Mitt kort blir ...",Temporary increase of e-commerce limit,"Hi there, Hi, I'm Sofia Ahonen. My card gets w...",Temporary increase of e-commerce limit,"Hi there, Hi, I'm [PERSON]. My card gets wrong...",Temporary increase of e-commerce limit|| there...,"temporary increase commerce limit, increase co..."
9,TKT-500009,en,Question about last month’s fees,"Hi, I'm Alex Ahonen. Not sure if this is on my...",Question about last month’s fees,"Hi, I'm Alex Ahonen. Not sure if this is on my...",Question about last month’s fees,"Hi, I'm [PERSON]. Not sure if this is on my si...","Question about last month’s fees|| , I'm [PERS...","card 4970 7040 tes

In [22]:
# Extract keywords from the combined text
keywords_list = prep.extract_keywords_batch(tickets_cleaned["combined_text"].tolist(), top_n=10)

# Store keywords as a comma-separated string
tickets_cleaned["keywords"] = [", ".join(kws) for kws in keywords_list]

# Verify
tickets_cleaned.head()

Extracting keywords:   2%|▎              | 1001/49800 [00:47<38:30, 21.12text/s]

[INFO] Processed 1000/49800 texts


Extracting keywords:   4%|▌              | 2003/49800 [01:50<57:45, 13.79text/s]

[INFO] Processed 2000/49800 texts


Extracting keywords:   6%|▉              | 3001/49800 [03:01<58:44, 13.28text/s]

...

Extracting keywords:  98%|█████████████▊| 49002/49800 [56:06<00:56, 14.04text/s]

[INFO] Processed 49000/49800 texts


Extracting keywords: 100%|██████████████| 49800/49800 [57:05<00:00, 14.54text/s]

[INFO] Keyword extraction completed for 49800 texts


,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked,combined_text,keywords
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting...",URGENT: Hitting rate limits on /transactions (...,"rate limits transactions error, limits transac..."
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti...",Question about KYC — documents keep getting re...,"kyc documents getting rejected, kyc documents ..."
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is...","Can’t pay online — keeps declining|| team, , I...","pay online keeps declining, pay online keeps, ..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ...","PRIO: Card missing, block now|| , , I'm [PERSO...","prio card missing block, prio card missing, bl..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ...","Tokenization failed during wallet setup|| , , ...","tokenization failed wallet setup, tokenization..."


In [7]:

max_words = tickets_cleaned["combined_text"].apply(lambda x: len(str(x).split())).max()
print("Maximum word count:", max_words)


Maximum word count: 554


In [23]:
tickets_cleaned.to_csv("tickets_cleaned.csv", index=False)

In [24]:
# Load from CSV into DataFrame
tickets_cleaned = pd.read_csv("tickets_cleaned.csv")

# Preview
tickets_cleaned.head(10)

,ticket_id,lang,subject_clean,body_clean,subject_translated,body_translated,subject_masked,body_masked,combined_text,keywords
0,TKT-500000,en,URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm Sam Heikkinen. My card keeps ge...",URGENT: Hitting rate limits on /transactions (...,"Hello, Hi, I'm [PERSON]. My card keeps getting...",URGENT: Hitting rate limits on /transactions (...,"rate limits transactions error, limits transac..."
1,TKT-500001,en,Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm Alex Mäkinen. My card keeps g...",Question about KYC — documents keep getting re...,"Hi team, Hi, I'm [PERSON]. My card keeps getti...",Question about KYC — documents keep getting re...,"kyc documents getting rejected, kyc documents ..."
2,TKT-500002,en,Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm Jamie Salonen. Not sure if th...",Can’t pay online — keeps declining,"Hi team, Hi, I'm [PERSON]. Not sure if this is...","Can’t pay online — keeps declining|| team, , I...","pay online keeps declining, pay online keeps, ..."
3,TKT-500003,sv,"PRIO: Kort saknas, spärra nu","Hallå, Hi, I'm Henrik Svensson. Jag har proble...","PRIO: Card missing, block now","Hello, Hi, I'm Henrik Svensson. I have problem...","PRIO: Card missing, block now","Hello, Hi, I'm [PERSON]. I have problems with ...","PRIO: Card missing, block now|| , , I'm [PERSO...","prio card missing block, prio card missing, bl..."
4,TKT-500004,en,Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm Sa Ahonen. Not sure if this is on...",Tokenization failed during wallet setup,"Hey, Hi, I'm [PERSON]. Not sure if this is on ...","Tokenization failed during wallet setup|| , , ...","tokenization failed wallet setup, tokenization..."
5,TKT-500005,en,"FYI: Sandbox ok, prod failing","Hello, Hi, I'm Jordan Mäkinen. I'm having trou...","FYI: Sandbox ok, prod failing","Hello, Hi, I'm Jordan Mäkinen. I'm having trou...","FYI: [PERSON], prod failing","Hello, Hi, I'm [PERSON]. I'm having trouble wi...","FYI: Sandbox ok, prod failing|| , , I'm [PERSO...","trouble integration sdk launch, trouble integr..."
6,TKT-500006,fi,PRIO: Miten kasvattaa API-kiintiötä?,"Hei, Hi, I'm Sanna Mäkinen. Ongelma liittyy ai...",PRIO: How to increase the API quota?,"Hi, Hi, I'm Sanna Mäkinen. The problem relates...",PRIO: How to increase the API quota?,"Hi, Hi, I'm [PERSON]. The problem relates to: ...","PRIO: How to increase the API quota?|| , , I'm...","prio increase api quota, increase api quota pr..."
7,TKT-500007,en,Declined transactions without reason?,"Hello, Hi, I'm Drew Laine. My card keeps getti...",Declined transactions without reason?,"Hello, Hi, I'm Drew Laine. My card keeps getti...",Declined transactions without reason?,"Hello, Hi, I'm [PERSON]. My card keeps getting...","Declined transactions without reason?|| , , I'...","card keeps getting declined, declined transact..."
8,TKT-500008,sv,Tillfällig höjning av e‑handelsgräns,"Hej där, Hi, I'm Sofia Ahonen. Mitt kort blir ...",Temporary increase of e-commerce limit,"Hi there, Hi, I'm Sofia Ahonen. My card gets w...",Temporary increase of e-commerce limit,"Hi there, Hi, I'm [PERSON]. My card gets wrong...",Temporary increase of e-commerce limit|| there...,"commerce limit card gets, increase commerce li..."
9,TKT-500009,en,Question about last month’s fees,"Hi, I'm Alex Ahonen. Not sure if this is on my...",Question about last month’s fees,"Hi, I'm Alex Ahonen. Not sure if this is on my...",Question about last month’s fees,"Hi, I'm [PERSON]. Not sure if this is on my si...","Question about last month’s fees|| , I'm [PERS...","fees sure tested d

In [25]:
# Write DataFrame to Postgres
tickets_cleaned.to_sql(
    name="ticket_preprocessed",   
    con=engine,
    if_exists="replace",           
    index=False,                   
    dtype=None                     
)

print("Data successfully written to 'ticket_preprocessed' table.")

Data successfully written to 'ticket_preprocessed' table.
